<a href="https://colab.research.google.com/github/Nagmashaik123/Section203-Group06/blob/main/Countries_Cities_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"CovidVaccine.csv",error_bad_lines=False)
df


In [ ]:
!pip install geopy

In [ ]:
import numpy as np
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['TweetLocation'] = (df['TweetLocation'].str.replace(emojisCollection, '').str.strip())

df['TweetLocation'] = df.TweetLocation.str.split(', ').str.get(0)
df['TweetLocation'] = df['TweetLocation'].replace(np.nan, 'Unknown', regex=True)
df['TweetLocation']

0                     San Antonio
1                         Unknown
2                         Unknown
3                          London
4                        Montreal
5                   United States
6                       Las Vegas
7                      Georgetown
8                        Portland
9                      Pittsburgh
10                        Unknown
11                      Gateshead
12                         Indore
13                        England
14                        Unknown
15                      Frankfort
16    Royaume-Uni. United Kingdom
17                         Canada
18                        Unknown
19                   Winter Haven
20                      Beckenham
21                        Unknown
22                         Orange
23                    District 13
24                     Washington
Name: TweetLocation, dtype: object

In [ ]:
from functools import partial
from geopy.geocoders import Nominatim
import re

countryList =[]
# Function to map the country for the corresponding city location
def checkCountry(city):
  geolocator = Nominatim(user_agent="COVID-19 vaccination attitudes")
  geocode = partial(geolocator.geocode, language="en")
  # passing the city through geocode api to return the location if city is not unknow
  if city != "Unknown" and not re.match(r'^[_\W]+$', city)  :
    location = geocode(city)   
    if location.address.find(", ") != -1:      
      country = str(location.address).split(", ")[-1] 
    elif location is None:
      country = "Unknown"   
    else:
      country = location.address  
  # If city is not known then setting country as Unknown        
  else:    
    country = "Unknown"   
  return country

    

In [ ]:
cities=[]
countries=[]
cities = df['TweetLocation'].tolist()
for city in cities:
    try:
        countries.append(checkCountry(city))  #Calling the checkCountry function
    except:
        countries.append('Unknown')

In [ ]:
# Zipping city and corresponding country together
dict_1=dict(zip(cities,countries))

In [ ]:
df['Country']=df['TweetLocation'].map(dict_1)
df

,TweetDate,TweetText,TweetAuthor,TweetLocation,TweetCountry,AllHashtags,Country
0,2021-03-16 17:04:39,Immediately after a needle related panic attac...,kstep38,Unknown,Unknown,['CovidVaccine'],Unknown
1,2021-03-16 17:04:27,Facebook is launching a new push to promote th...,MikeMck84268,Huddersfield,United Kingdom,"['Facebook', '100daysofcode', 'COVID19Vaccine'...",United Kingdom
2,2021-03-16 17:04:18,Last Friday's Urdu Jumu'ah khutbah was also re...,MakkiMasjid_Mcr,Manchester,United Kingdom,"['COVID19', 'vaccine', 'Covid_19', 'CovidVacci...",United Kingdom
3,2021-03-16 17:04:07,"So while the ""Last guy"" was on TV saying Covid...",Deemoney521,Unknown,Unknown,['CovidVaccine'],Unknown
4,2021-03-16 17:03:48,"A year of so much sacrifice and loss, but now....",lilkioko,Chicago,United States,"['vaccinated', 'ScienceRocks']",United States
...,...,...,...,...,...,...,...
9995,2021-03-14 09:17:35,#Marr @Channel4News @SkyNews \n\nThink you're ...,WaronSpeech,uk,United Kingdom,"['Marr', 'GAVI', 'vaccines', 'billgates', 'COV...",United Kingdom
9996,2021-03-14 09:17:34,#Marr @Channel4News @SkyNews \n\nThink you're ...,WaronSpeech,uk,United Kingdom,"['Marr', 'GAVI', 'vaccines', 'billgates', 'COV...",United Kingdom
9997,2021-03-14 09:17:34,#Marr think you're clever\nhttps://t.co/EbQ79s...,WaronSpeech,uk,United Kingdom,"['Marr', 'GAVI', 'vaccines', 'billgates', 'COV...",United Kingdom
9998,2021-03-14 09:17:32,Interactable #heatmap showing number of new #C...,hangangmagazine,Seoul,South Korea,"['heatmap', 'COVIDVaccinations', 'Korea', 'cor...",South Korea


In [ ]:
df.to_csv('CovidVaccineWorldWide.csv')